In [3]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch.optim as optim
from sklearn.utils import shuffle
from torch.autograd import Variable

In [4]:
df=pd.read_csv('ratings.csv',names=['userID','movieID','rating','time'])
df.drop('time',axis=1,inplace=True)
users=[k for k,v in df['userID'].value_counts().iteritems() if v>2]
movies=[k for k,v in df['movieID'].value_counts().iteritems() if v>10]
df=df[(df['userID'].isin(users)) & (df['movieID'].isin(movies))]
df=df.pivot(index='userID',columns='movieID',values='rating')
print (df.shape)

(240447, 14277)


In [5]:
train_df=df.iloc[0:220000]
test_df=df.iloc[230000:].reset_index(drop=True)
val_df=df.iloc[220000:230000].reset_index(drop=True)

In [7]:
class VAE(nn.Module):
    def __init__(self, input_size, hidden_size_1, hidden_size_2):
        super(VAE,self).__init__()
        self.input_size=input_size
        self.hidden_size_1=hidden_size_1
        self.hidden_size_2=hidden_size_2
        #self.training=training
        
        self.encoder_linear_l1=nn.Linear(input_size, hidden_size_1)
        self.encoder_linear_l2=nn.Linear(hidden_size_1, 2*hidden_size_2)
        
        self.decode_linear_l1=nn.Linear(hidden_size_2, hidden_size_1)
        self.decode_linear_l2=nn.Linear(hidden_size_1, input_size)
        
        self.sigmoid=nn.LogSigmoid()
        
    def forward(self, input_ratings):
        mu,logvar=self.encode(input_ratings)
        param=self.reparameterize(mu,logvar)
        decoded=self.sigmoid(self.decode(param))
        return decoded,mu,logvar
    
    def encode(self, input_ratings):
        enc_out=F.relu(self.encoder_linear_l1(input_ratings))
        enc_out=self.encoder_linear_l2(enc_out)
        log_var=enc_out[:,self.hidden_size_2:]
        mu=enc_out[:,:self.hidden_size_2]
        return mu, log_var
    
    def reparameterize(self, mu, logvar):
        if self.training:
            std=torch.exp(0.5*logvar)
            eps = torch.randn_like(std)
            return eps.mul(std).add_(mu)
        else:
            return mu
    
    def decode(self,param):
        dec_out=F.relu(self.decode_linear_l1(param))
        dec_out=self.decode_linear_l2(dec_out)
        return dec_out


In [9]:
def loss_criterion(decoded,input_ratings,mu,logvar,annealing_coef):
    bce_loss=-torch.mean(torch.sum(input_ratings*decoded,-1))
    kl_divg=-0.5*torch.mean(torch.sum(1 + logvar - mu.pow(2) - logvar.exp(), dim=1))
    return bce_loss+annealing_coef*kl_divg

In [10]:
def train_minibatch(input_ratings, mask, vae, optimizer):
    vae.train()
    optimizer.zero_grad()
    input_ratings=input_ratings.type(torch.cuda.FloatTensor)
    #input_ratings=F.normalize(input_ratings,p=1)
    output_ratings,mu,logvar=vae(input_ratings)
    loss=loss_criterion(output_ratings,input_ratings,mu,logvar,annealing_coef=0.2)
    loss.backward()
    optimizer.step()
    return loss

In [11]:
def validation(input_ratings, mask, vae):
    with torch.no_grad():
        vae.eval()
        hit_list=[]
        input_ratings=input_ratings.type(torch.cuda.FloatTensor)
        #input_ratings=F.normalize(input_ratings,p=1)
        for i in range(input_ratings.size(0)):
            idx=torch.nonzero(mask[i])
            if (idx.size(0)>5):
                idx=idx[-1]
                input_ratings[i][idx]=0
                output_ratings=vae(input_ratings[i].unsqueeze(0))[0]
                _,indices=torch.topk(output_ratings.squeeze(0),50)
                if(idx in indices):
                    hit_list.append(1)
                else:
                    hit_list.append(0)
        return np.mean(hit_list)

In [12]:
vae=VAE(input_size=train_df.shape[1], hidden_size_1=600, hidden_size_2=200)
optimizer=optim.Adam(vae.parameters())
device=torch.device('cuda')
vae=vae.to(device)

In [13]:
val_mask=val_df.copy()
val_mask[~val_mask.isnull()] = 1  # not nan
val_mask[val_mask.isnull()] = 0   # nan
val_df[val_df.isnull()] = 0   # nan
val_df[val_df<=3]=0
val_df[val_df>3]=1

In [14]:
input_users_val=torch.from_numpy(val_df.values).to(device).detach()
mask_val=torch.from_numpy(val_mask.values).to(device).detach()

In [ ]:
checkpoint = torch.load('model_VAE.pth')
autorec.load_state_dict(checkpoint['autorec_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
autorec.eval()

In [15]:
num_batches=10
val_benchmark=0



for batch in range(0,num_batches):
    running_loss=0
    #train_df = shuffle(train_df)
    for i in range(0,train_df.shape[0],100):
        #print(i)
        tdf=(train_df.iloc[i:i+100].copy()).sample(frac=1)
        train_mask=tdf.copy()
        train_mask[~train_mask.isnull()] = 1  # not nan
        train_mask[train_mask.isnull()] = 0   # nan
        tdf[tdf.isnull()] = 0
        tdf[tdf<=3]=0
        tdf[tdf>3]=1
        input_users=Variable(torch.from_numpy(tdf.values)).to(device)
        input_mask=torch.from_numpy(train_mask.values).to(device)
        loss=train_minibatch(input_users, input_mask, vae, optimizer)
        running_loss+=loss.item()
        if (i)%44000==0:
            val_hr=validation(input_users_val, mask_val, vae)
            print ('Batch: {} | Step: {}/{} | Training Loss: {} | Validation hit ratio: {}'.format(batch+1,int(i/44000)+1,5,running_loss,round(val_hr,4) ))
            running_loss=0
            if(val_hr>val_benchmark):
                print ('%---Saving the model---%')
                torch.save({
                    'step':i+1,
                    'autorec_state_dict': vae.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'batch':batch,
                    'loss':val_hr
                    },'model_VAE.pth')
                val_benchmark=val_hr

Batch: 1 | Step: 1/5 | Training Loss: 2.618682622909546 | Validation hit ratio: 0.0028
%---Saving the model---%
Batch: 1 | Step: 2/5 | Training Loss: 181.6016069613397 | Validation hit ratio: 0.1438
%---Saving the model---%
Batch: 1 | Step: 3/5 | Training Loss: 18.916725856717676 | Validation hit ratio: 0.1386
Batch: 1 | Step: 4/5 | Training Loss: 6.533714386168867 | Validation hit ratio: 0.1398
Batch: 1 | Step: 5/5 | Training Loss: 3.1022668709338177 | Validation hit ratio: 0.1378
Batch: 2 | Step: 1/5 | Training Loss: 0.0006280505331233144 | Validation hit ratio: 0.1367
Batch: 2 | Step: 2/5 | Training Loss: 1.1665297344734427 | Validation hit ratio: 0.1378
Batch: 2 | Step: 3/5 | Training Loss: 0.42711660130589735 | Validation hit ratio: 0.1386
Batch: 2 | Step: 4/5 | Training Loss: 0.3233424207137432 | Validation hit ratio: 0.1374
Batch: 2 | Step: 5/5 | Training Loss: 0.06828319674968952 | Validation hit ratio: 0.1374
Batch: 3 | Step: 1/5 | Training Loss: 0.00011142828589072451 | Valid

KeyboardInterrupt: 